# **K Means**

This notebook implements k Means from scratch

Import libraries:

In [11]:
import numpy as np

We are going to follow three steps in writing our kMeans algorithms:


1.   Define a function that computes distances and assign clusters
2.   Define a function that computes new centroid
3.   Define a function that combines the two and iterates until convergence




In [12]:
def distance_and_assignment(X, centroid_matrix, distance = False):

  """
  Arguments:
  ---------

  - X: a Numpy array of dimension (n x p), where n is the sample size and 
       p is the number of variables (indipendent)

  - centroid_matrix: a Numpy array of dimension (k x p), where k is the 
                     cluster size and p the number of variables

  - distance: (logical), it specifies whether the user want to see the 
              distance matrix or not

  Returns:
  -------

  Case 1: if distance == True
          - distance_matrix, assigned_clusters: the distance matrix and
            the assigned clusters

  Case 2: if distance == Fales
          - assigned_clusters: the assigned clusters
  """
  ## to guarantee that number columns matches in both X and centroid matrix
  if X.shape[1] != centroid_matrix.shape[1]:
    return "pairwise vectors do not match!"
  else:
    n, k = X.shape[0], centroid_matrix.shape[0]
    X_sqr = X**2
    cm_sq = centroid_matrix**2
    X_dot = X_sqr.sum(axis = 1).reshape((n,1))*np.ones(shape = (1, k))
    cm_dot = cm_sq.sum(axis = 1)*np.ones(shape = (n, 1))
    distance_matrix = np.sqrt(X_dot + cm_dot - 2*X.dot(centroid_matrix.T))
    assigned_clusters = np.argmin(distance_matrix, axis = 1)
    if distance == True:
      return distance_matrix, assigned_clusters  
    return assigned_clusters

In [13]:
def centroids_computations(X, assigned_clusters, k):
  
  """
  Arguments:
  ---------

  - X: a Numpy array of dimension (n x p), where n is the sample size and 
      p is the number of variables (indipendent)
  
  - assigned_clusters: a Numpy array of dimension (n,) of the assigned 
                       clusters. cluster = {0, 1, ..., k-1}

  - k: (integer) cluster size

  Returns:
  -------

  new_centroids: a Numpy array of dimension (k x p) of new cluster centroid, 
                 where k is the cluster size and p the number of variables
  """

  new_centroids = []
  for c in range(k):
    x1 = X[assigned_clusters == c]
    x1 = np.mean(x1, axis = 0)
    x1[np.isnan(x1)] = 0
    new_centroids.append(x1)
  return np.array(new_centroids)

In [14]:
def kMeans(X, distance_and_assignment, centroids_computations, centroid_matrix, 
           k, distance = False):
  
  """
  Arguments:
  ---------

   - X: a Numpy array of dimension (n x p), where n is the sample size and 
      p is the number of variables (indipendent)

   - distance_and_assignment: a function that computes distance and perform 
                             cluster assignment

   - centroids_computations: a function that computes centroids

   - centroid_matrix: the initial centroid matrix

   - k: (integer) cluster size

   - distance: (logical), it specifies whether the user want to see the 
              distance matrix or not
                             

  Returns:
  -------

  present_cluster: optimal cluster

  iter: (integer) number of iterations until convergence
  """
  
  prev_clusters = distance_and_assignment(X, centroid_matrix, distance = False)
  new_cent = centroids_computations(X, prev_clusters, k)
  centroid_matrix = new_cent
  present_clusters = distance_and_assignment(X, centroid_matrix, distance = False)
  iter = 2
  while not np.alltrue(prev_clusters == present_clusters):
    prev_clusters = present_clusters
    new_cent = centroids_computations(X, prev_clusters, k)
    centroid_matrix = new_cent
    present_clusters = distance_and_assignment(X, centroid_matrix, distance = False)
    iter+=1
  return present_clusters, "Number of iteration until convergence = " + str(iter)

## Application

In [15]:
X = np.genfromtxt("Data.tsv", delimiter="\t")
init_cent = np.array([[1.03800476, 0.09821729, 1.0469454, 1.58046376],
                      [0.18982966, -1.97355361, 0.70592084, 0.3957741],
                      [1.2803405, 0.09821729, 0.76275827, 1.44883158]])

In [16]:
optim_clust, j = kMeans(X, distance_and_assignment, centroids_computations, centroid_matrix = init_cent, 
           k = 3, distance = False)
optim_clust, j

(array([1, 2, 2, 2, 1, 1, 1, 1, 1, 2, 0, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 0,
        1, 2, 1, 2, 1, 2, 1, 1, 1, 2, 2, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 0,
        2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0,
        2, 2, 1, 0, 2, 1, 1, 1, 1, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0,
        0, 0, 2, 2, 0, 2, 2, 0, 2, 0, 0, 2, 2, 2, 0, 0, 2, 2, 2, 0, 2, 0,
        0, 0, 0, 2, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2,
        0, 0, 0, 2, 2, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 2, 2, 0]),
 'Number of iteration until convergence = 8')

In [17]:
optim_clust.tofile('kmeans_output.tsv', sep = '\t')

### Comparing with Scikit learn

In [18]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, random_state=0, init=init_cent).fit(X)

kmeans.labels_

/home/eviofekeze/anaconda3/envs/dsb/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1307: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  super()._check_params(X)


array([1, 2, 2, 2, 1, 1, 1, 1, 1, 2, 0, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 0,
       1, 2, 1, 2, 1, 2, 1, 1, 1, 2, 2, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 0,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0,
       2, 2, 1, 0, 2, 1, 1, 1, 1, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0,
       0, 0, 2, 2, 0, 2, 2, 0, 2, 0, 0, 2, 2, 2, 0, 0, 2, 2, 2, 0, 2, 0,
       0, 0, 0, 2, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2,
       0, 0, 0, 2, 2, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 2, 2, 0], dtype=int32)

In [19]:
kmeans.n_iter_ ## Number of iterations

8

### **Are they Equal?**

In [20]:
optim_clust == kmeans.labels_

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,